막막한 모델 구현을 쉽게 이해하기 위해, baseline의 구조를 하나의 흐름으로 펴는 작업을 수행한다.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
import time
import random
import logging
import json
import tqdm

from sklearn.model_selection import train_test_split
from torch.nn import MSELoss
from torch.optim import SGD, Adam
from torch.utils.data import TensorDataset, DataLoader, Dataset

다양한 모델을 위한 args들이 함수의 입력으로 계속 따라붙으면서, baseline의 구조가 복잡하게 느껴진다.

우선 FM 모델 하나만의 흐름으로 확인하자.  
모델 흐름 이해에는 도움되지 않는 args들을 미리 지정해둔다.

In [2]:
data_path = '/opt/ml/data/'
saved_model_path ='./saved_models'
model='FM' # ['FM', 'FFM', 'NCF', 'WDN', 'DCN', 'CNN_FM', 'DeepCoNN']
data_shuffle=True
test_size=0.2
seed=42
use_best_model=True

batch_size = 1024
epochs = 10
lr = 1e-3
loss_fn = 'RMSE'
optimizer = 'ADAM'
weight_decay = 1e-6

device = 'cuda'
embed_dim = 16
dropout = 0.2
mlp_dims = (16,16)
num_layers = 3

미션1 EDA 파일과 실제 데이터가 다른 점이 꽤 존재해서, 이들을 잘 처리해주어야 할 것 같다.

In [3]:
# def context_data_load(args):
######################## DATA LOAD
users = pd.read_csv(data_path + 'users.csv')
books = pd.read_csv(data_path + 'books.csv')
train = pd.read_csv(data_path + 'train_ratings.csv')
test = pd.read_csv(data_path + 'test_ratings.csv')
sub = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
# id와 isbn(책 번호)를 쉬운 index로 바꾼다.
ids = pd.concat([train['user_id'], sub['user_id']]).unique()
isbns = pd.concat([train['isbn'], sub['isbn']]).unique()

In [5]:
idx2user = {idx:id for idx, id in enumerate(ids)}
idx2isbn = {idx:isbn for idx, isbn in enumerate(isbns)}

user2idx = {id:idx for idx, id in idx2user.items()}
isbn2idx = {isbn:idx for idx, isbn in idx2isbn.items()}

# id와 isbn을 간단한 index로 변환
train['user_id'] = train['user_id'].map(user2idx)
sub['user_id'] = sub['user_id'].map(user2idx)
test['user_id'] = test['user_id'].map(user2idx)
users['user_id'] = users['user_id'].map(user2idx)

train['isbn'] = train['isbn'].map(isbn2idx)
sub['isbn'] = sub['isbn'].map(isbn2idx)
test['isbn'] = test['isbn'].map(isbn2idx)
books['isbn'] = books['isbn'].map(isbn2idx)

In [6]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

In [7]:
# def process_context_data(users, books, ratings1, ratings2):
# location 분리
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())
# users = users.drop(['location'], axis=1)

users = users.replace(['n/a','na','nan',''], np.nan)

ratings = pd.concat([train, test]).reset_index(drop=True)
print(ratings.shape)

(383494, 3)


In [8]:
users[users['location_country'].isna()]

,user_id,location,age,location_city,location_state,location_country
2,13.0,"n/a, n/a, n/a",NaN,NaN,NaN,NaN
6,13426.0,"ottawa, ,",NaN,ottawa,NaN,NaN
32,17.0,"seattle, ,",27.0,seattle,NaN,NaN
49,3087.0,"albuquerque, ,",NaN,albuquerque,NaN,NaN
72,9776.0,"humble, ,",38.0,humble,NaN,NaN
...,...,...,...,...,...,...
67797,59467.0,"lisbon, maine,",36.0,lisbon,maine,NaN
67929,59607.0,"houston, ,",NaN,houston,NaN,NaN
67930,59608.0,"sammamish, ,",NaN,sammamish,NaN,NaN
68058,59762.0,"calgary, ,",NaN,calgary,NaN,NaN


In [9]:
users.isna().sum()

user_id                23
location                0
age                 27833
location_city         101
location_state       3185
location_country     2122
dtype: int64

In [10]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

/tmp/ipykernel_2139/3274669727.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
/tmp/ipykernel_2139/3274669727.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]


In [11]:
location_list

['ottawa, ontario, canada',
 'seattle, washington, usa',
 'albuquerque, new mexico, usa',
 'humble, texas, usa',
 'aloha, oregon, usa',
 'pearland, texas, usa',
 'sarasota, florida, usa',
 'west springfield, massachusetts, usa',
 'west linn, oregon, usa',
 'north fort myers, florida, usa',
 'coconut grove, florida, usa',
 'mercer island, washington, usa',
 'mercer island, washington, usa',
 'de soto, missouri, usa',
 'vigo, pontevedra, spain',
 'carmichael, california, usa',
 'chicago, illinois, usa',
 'kuala lumpur, kuala lumpur, malaysia',
 'tucson, arizona, usa',
 'binghamton, new york, usa',
 'yarmouth, nova scotia, canada',
 'patterson lakes, victoria, australia',
 'tucson, arizona, usa',
 'manchester, england, united kingdom',
 'cotati, california, usa',
 'tualatin, oregon, usa',
 'san francisco, california, usa',
 'chesterfield, missouri, usa',
 'brooklyn, new york, usa',
 'oxford, england, united kingdom',
 'anchorage, alaska, usa',
 'chicago, illinois, usa',
 'molalla, oregon,

In [12]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

In [13]:
users.isna().sum()

user_id                23
location                0
age                 27833
location_city         101
location_state       1091
location_country      273
dtype: int64

In [14]:
print(len(users),len(user2idx))

68092 68069


In [15]:
# 인덱싱 처리된 데이터 조인
context_df = ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
train_df = train.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
test_df = test.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')

user_id와 isbn이 간단한 index로 바뀐 것을 확인할 수 있다.

In [16]:
train_df

,user_id,isbn,rating,location,age,location_city,location_state,location_country,category,publisher,language,book_author
0,0,0,4,"timmins, ontario, canada",NaN,timmins,ontario,canada,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright
1,1,0,7,"toronto, ontario, canada",30.0,toronto,ontario,canada,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright
2,2,0,8,"kingston, ontario, canada",NaN,kingston,ontario,canada,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright
3,3,0,8,"comber, ontario, canada",NaN,comber,ontario,canada,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright
4,4,0,9,"guelph, ontario, canada",NaN,guelph,ontario,canada,['Actresses'],HarperFlamingo Canada,en,Richard Bruce Wright
...,...,...,...,...,...,...,...,...,...,...,...,...
306790,6313,129772,7,"pismo beach, california, usa",28.0,pismo beach,california,usa,NaN,Simon & Schuster Audio,NaN,David Gardner
306791,1879,129773,6,"dallas, texas, usa",33.0,dallas,texas,usa,['Humor'],Pocket Books,en,P.J. O'Rourke
306792,1879,129774,7,"dallas, texas, usa",33.0,dallas,texas,usa,NaN,Lone Star Books,NaN,Claude Dooley
306793,1879,129775,7,"dallas, texas, usa",33.0,dallas,texas,usa,['Fiction'],Kqed Books,en,Jeremy Lloyd


age와 Category, Language등 결측값이 존재하는 것을 확인할 수 있다.

In [189]:
# 이제 나머지 인덱스들도 숫자로 변환하는 작업을 수행한다.
# 결측치도 숫자로 바꿔버린다.
loc_city2idx = {v:k for k,v in enumerate(context_df['location_city'].unique())}
loc_state2idx = {v:k for k,v in enumerate(context_df['location_state'].unique())}
loc_country2idx = {v:k for k,v in enumerate(context_df['location_country'].unique())}
loc_city2idx[np.nan] = np.nan
loc_state2idx[np.nan] = np.nan
loc_country2idx[np.nan] = np.nan
loc_country2idx
# loc_state2idx

In [190]:
train_df['location_city'] = train_df['location_city'].map(loc_city2idx)
train_df['location_state'] = train_df['location_state'].map(loc_state2idx)
train_df['location_country'] = train_df['location_country'].map(loc_country2idx)
test_df['location_city'] = test_df['location_city'].map(loc_city2idx)
test_df['location_state'] = test_df['location_state'].map(loc_state2idx)
test_df['location_country'] = test_df['location_country'].map(loc_country2idx)

# age의 결측치는 평균으로 채운다. 
# train_df['age'] = train_df['age'].fillna(int(train_df['age'].mean()))
# train_df['age'] = train_df['age'].apply(age_map)
# test_df['age'] = test_df['age'].fillna(int(test_df['age'].mean()))
# test_df['age'] = test_df['age'].apply(age_map)

In [191]:
# book 파트 인덱싱
category2idx = {v:k for k,v in enumerate(context_df['category'].unique())}
publisher2idx = {v:k for k,v in enumerate(context_df['publisher'].unique())}
language2idx = {v:k for k,v in enumerate(context_df['language'].unique())}
author2idx = {v:k for k,v in enumerate(context_df['book_author'].unique())}
category2idx[np.nan] = np.nan
publisher2idx[np.nan] = np.nan
language2idx[np.nan] = np.nan
author2idx[np.nan] = np.nan

In [192]:
train_df['category'] = train_df['category'].map(category2idx)
train_df['publisher'] = train_df['publisher'].map(publisher2idx)
train_df['language'] = train_df['language'].map(language2idx)
train_df['book_author'] = train_df['book_author'].map(author2idx)
test_df['category'] = test_df['category'].map(category2idx)
test_df['publisher'] = test_df['publisher'].map(publisher2idx)
test_df['language'] = test_df['language'].map(language2idx)
test_df['book_author'] = test_df['book_author'].map(author2idx)

모든 데이터가 숫자로 바뀐 것을 확인할 수 있다.

In [193]:
train_df
test_df

,user_id,isbn,rating,age,location_city,location_state,location_country,category,publisher,language,book_author
0,13,0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,13426,0,0,NaN,309.0,NaN,NaN,0.0,0.0,0.0,0.0
2,26761,1,0,40.0,309.0,0.0,0.0,1.0,1.0,0.0,1.0
3,16495,2,0,30.0,195.0,14.0,1.0,2.0,2.0,0.0,2.0
4,6225,3,0,39.0,2716.0,92.0,1.0,3.0,3.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
76694,7728,149565,0,39.0,3086.0,5.0,1.0,73.0,310.0,0.0,1275.0
76695,47785,149566,0,37.0,323.0,104.0,18.0,985.0,108.0,2.0,10774.0
76696,4209,149567,0,NaN,20.0,18.0,1.0,3.0,7792.0,0.0,62056.0
76697,40779,149568,0,48.0,210.0,66.0,13.0,NaN,5050.0,NaN,62057.0


In [194]:
temp_columns = train_df.columns
temp_columns

Index(['user_id', 'isbn', 'rating', 'age', 'location_city', 'location_state',
       'location_country', 'category', 'publisher', 'language', 'book_author'],
      dtype='object')

In [195]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer_mice = IterativeImputer(random_state=9)
train_df = pd.DataFrame(imputer_mice.fit_transform(train_df)).applymap(lambda x:round(x))
test_df = pd.DataFrame(imputer_mice.fit_transform(test_df)).applymap(lambda x:round(x))
train_df.columns = temp_columns
test_df.columns = temp_columns

In [196]:
train_df

,user_id,isbn,rating,age,location_city,location_state,location_country,category,publisher,language,book_author
0,0,0,4,37,0,0,0,0,0,0,0
1,1,0,7,30,1,0,0,0,0,0,0
2,2,0,8,37,2,0,0,0,0,0,0
3,3,0,8,37,3,0,0,0,0,0,0
4,4,0,9,37,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
306790,6313,129772,7,28,1606,5,1,324,2171,0,1272
306791,1879,129773,6,33,19,10,1,7,222,0,69
306792,1879,129774,7,33,19,10,1,796,10406,0,54713
306793,1879,129775,7,33,19,10,1,3,6387,0,54714


In [197]:
idx = {
    "loc_city2idx":loc_city2idx,
    "loc_state2idx":loc_state2idx,
    "loc_country2idx":loc_country2idx,
    "category2idx":category2idx,
    "publisher2idx":publisher2idx,
    "language2idx":language2idx,
    "author2idx":author2idx,
}

In [198]:
context_train, context_test = train_df, test_df
field_dims = np.array([len(user2idx), len(isbn2idx),
                        6, len(idx['loc_city2idx']), len(idx['loc_state2idx']), len(idx['loc_country2idx']),
                        len(idx['category2idx']), len(idx['publisher2idx']), len(idx['language2idx']), len(idx['author2idx'])], dtype=np.uint32)

data = {
        'train':context_train,
        'test':context_test.drop(['rating'], axis=1),
        'field_dims':field_dims,
        'users':users,
        'books':books,
        'sub':sub,
        'idx2user':idx2user,
        'idx2isbn':idx2isbn,
        'user2idx':user2idx,
        'isbn2idx':isbn2idx,
        }

In [199]:
field_dims

array([ 68069, 149570,      6,  12265,   1594,    344,   4293,  11572,
           27,  62059], dtype=uint32)

In [200]:
# def context_data_split(args, data):
X_train, X_valid, y_train, y_valid = train_test_split(
                                                    data['train'].drop(['rating'], axis=1),
                                                    data['train']['rating'],
                                                    test_size=test_size,
                                                    random_state=seed,
                                                    shuffle=True
                                                    )
data['X_train'], data['X_valid'], data['y_train'], data['y_valid'] = X_train, X_valid, y_train, y_valid

In [201]:
# def context_data_loader(args, data):
train_dataset = TensorDataset(torch.LongTensor(data['X_train'].values), torch.LongTensor(data['y_train'].values))
valid_dataset = TensorDataset(torch.LongTensor(data['X_valid'].values), torch.LongTensor(data['y_valid'].values))
test_dataset = TensorDataset(torch.LongTensor(data['test'].values))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=data_shuffle)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=data_shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

data['train_dataloader'], data['valid_dataloader'], data['test_dataloader'] = train_dataloader, valid_dataloader, test_dataloader

In [202]:
class Setting:
    @staticmethod
    def seed_everything(seed):
        '''
        [description]
        seed 값을 고정시키는 함수입니다.

        [arguments]
        seed : seed 값
        '''
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    def __init__(self):
        now = time.localtime()
        now_date = time.strftime('%Y%m%d', now)
        now_hour = time.strftime('%X', now)
        save_time = now_date + '_' + now_hour.replace(':', '')
        self.save_time = save_time

    def get_log_path(self, model):
        '''
        [description]
        log file을 저장할 경로를 반환하는 함수입니다.

        [arguments]
        args : argparse로 입력받은 args 값으로 이를 통해 모델의 정보를 전달받습니다.

        [return]
        path : log file을 저장할 경로를 반환합니다.
        이 때, 경로는 log/날짜_시간_모델명/ 입니다.
        '''
        path = f'./log/{self.save_time}_{model}/'
        return path

    def get_submit_filename(self, model):
        '''
        [description]
        submit file을 저장할 경로를 반환하는 함수입니다.

        [arguments]
        args : argparse로 입력받은 args 값으로 이를 통해 모델의 정보를 전달받습니다.

        [return]
        filename : submit file을 저장할 경로를 반환합니다.
        이 때, 파일명은 submit/날짜_시간_모델명.csv 입니다.
        '''
        path = self.make_dir("./submit/")
        filename = f'{path}{self.save_time}_{model}.csv'
        return filename

    def make_dir(self,path):
        '''
        [description]
        경로가 존재하지 않을 경우 해당 경로를 생성하며, 존재할 경우 pass를 하는 함수입니다.

        [arguments]
        path : 경로

        [return]
        path : 경로
        '''
        if not os.path.exists(path):
            os.makedirs(path)
        else:
            pass
        return path


class Logger:
    def __init__(self, model, path):
        """
        [description]
        log file을 생성하는 클래스입니다.

        [arguments]
        args : argparse로 입력받은 args 값으로 이를 통해 모델의 정보를 전달받습니다.
        path : log file을 저장할 경로를 전달받습니다.
        """
        self.model = model
        self.path = path

        self.logger = logging.getLogger()
        self.logger.setLevel(logging.INFO)
        self.formatter = logging.Formatter('[%(asctime)s] - %(message)s')

        self.file_handler = logging.FileHandler(self.path+'train.log')
        self.file_handler.setFormatter(self.formatter)
        self.logger.addHandler(self.file_handler)

    def log(self, epoch, train_loss, valid_loss):
        '''
        [description]
        log file에 epoch, train loss, valid loss를 기록하는 함수입니다.
        이 때, log file은 train.log로 저장됩니다.

        [arguments]
        epoch : epoch
        train_loss : train loss
        valid_loss : valid loss
        '''
        message = f'epoch : {epoch}/{epochs} | train loss : {train_loss:.3f} | valid loss : {valid_loss:.3f}'
        self.logger.info(message)

    def close(self):
        '''
        [description]
        log file을 닫는 함수입니다.
        '''
        self.logger.removeHandler(self.file_handler)
        self.file_handler.close()

# args는 초반에 다 지정해버렸기 때문에, 이 함수는 필요없다.
#     def save_args(self):
#         '''
#         [description]
#         model에 사용된 args를 저장하는 함수입니다.
#         이 때, 저장되는 파일명은 model.json으로 저장됩니다.
#         '''
#         argparse_dict = self.args.__dict__

#         with open(f'{self.path}/model.json', 'w') as f:
#             json.dump(argparse_dict,f,indent=4)

    def __del__(self):
        self.close()


In [203]:
setting = Setting()

log_path = setting.get_log_path(model)
setting.make_dir(log_path)

logger = Logger(model, log_path)
# logger.save_args()

In [204]:
# factorization을 통해 얻은 feature를 embedding 합니다.
class FeaturesEmbedding(nn.Module):
    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)


    def forward(self, x: torch.Tensor):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)


# FM모델 등에서 활용되는 선형 결합 부분을 정의합니다.
class FeaturesLinear(nn.Module):
    def __init__(self, field_dims: np.ndarray, output_dim: int=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)


    def forward(self, x: torch.Tensor):
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias


# feature 사이의 상호작용을 효율적으로 계산합니다.
class FactorizationMachine(nn.Module):
    def __init__(self, reduce_sum:bool=True):
        super().__init__()
        self.reduce_sum = reduce_sum


    def forward(self, x: torch.Tensor):
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [205]:
class FactorizationMachineModel(nn.Module):
    def __init__(self, data):
        super().__init__()
        self.field_dims = data['field_dims']
        self.embedding = FeaturesEmbedding(self.field_dims, embed_dim)
        self.linear = FeaturesLinear(self.field_dims)
        self.fm = FactorizationMachine(reduce_sum=True)


    def forward(self, x: torch.Tensor):
        x = self.linear(x) + self.fm(self.embedding(x))
        # return torch.sigmoid(x.squeeze(1))
        return x.squeeze(1)


In [206]:
# def models_load(args, data):
model = FactorizationMachineModel(data).to(device)

In [207]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.eps = 1e-6
    def forward(self, x, y):
        criterion = MSELoss()
        loss = torch.sqrt(criterion(x, y)+self.eps)
        return loss

In [208]:
def valid(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0
    batch = 0

    for idx, data in enumerate(dataloader['valid_dataloader']):
        x, y = data[0].to(device), data[1].to(device)
        y_hat = model(x)
        loss = loss_fn(y.float(), y_hat)
        total_loss += loss.item()
        batch +=1
    valid_loss = total_loss/batch
    return valid_loss



In [209]:
def train(model, dataloader, logger, setting):
    print(model)
    minimum_loss = 999999999
    loss_fn = RMSELoss()
    optimizer = Adam(model.parameters(), lr=lr)
    for epoch in tqdm.tqdm(range(epochs)):
        model.train()
        total_loss = 0
        batch = 0

        for idx, data in enumerate(dataloader['train_dataloader']):
            x, y = data[0].to(device), data[1].to(device)
            y_hat = model(x)
            loss = loss_fn(y.float(), y_hat)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            batch +=1
        valid_loss = valid(model, dataloader, loss_fn)
        print(f'Epoch: {epoch+1}, Train_loss: {total_loss/batch:.3f}, valid_loss: {valid_loss:.3f}')
        logger.log(epoch=epoch+1, train_loss=total_loss/batch, valid_loss=valid_loss)
        if minimum_loss > valid_loss:
            minimum_loss = valid_loss
            os.makedirs(saved_model_path, exist_ok=True)
            # torch.save(state_dict(), f'{saved_model_path}/{setting.save_time}_{model}_model.pt')
    logger.close()
    return model

In [210]:
model = train(model, data, logger, setting)

  0%|          | 0/10 [00:00<?, ?it/s]

FactorizationMachineModel(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(309799, 16)
  )
  (linear): FeaturesLinear(
    (fc): Embedding(309799, 1)
  )
  (fm): FactorizationMachine()
)


 10%|█         | 1/10 [00:04<00:36,  4.04s/it]

Epoch: 1, Train_loss: 4.774, valid_loss: 2.851


 20%|██        | 2/10 [00:08<00:32,  4.03s/it]

...

Epoch: 8, Train_loss: 1.466, valid_loss: 2.488


 90%|█████████ | 9/10 [00:36<00:04,  4.10s/it]

Epoch: 9, Train_loss: 1.418, valid_loss: 2.504


100%|██████████| 10/10 [00:40<00:00,  4.09s/it]

Epoch: 10, Train_loss: 1.381, valid_loss: 2.523


In [27]:
def test(model, dataloader, setting):
    predicts = list()
    # if use_best_model == True:
    #     model.load_state_dict(torch.load(f'./saved_models/{setting.save_time}_{model}_model.pt'))
    # else:
    #     pass
    model.eval()

    for idx, data in enumerate(dataloader['test_dataloader']):
        x = data[0].to(device)
        y_hat = model(x)
        predicts.extend(y_hat.tolist())
    return predicts


In [28]:
######################## SAVE PREDICT
print(f'--------------- SAVE {model} PREDICT ---------------')
submission = pd.read_csv(data_path + 'sample_submission.csv')
submission['rating'] = test(model, data, setting)
submission.to_csv('result', index=False)


--------------- SAVE FactorizationMachineModel(
  (embedding): FeaturesEmbedding(
    (embedding): Embedding(309818, 16)
  )
  (linear): FeaturesLinear(
    (fc): Embedding(309818, 1)
  )
  (fm): FactorizationMachine()
) PREDICT ---------------


FileNotFoundError: [Errno 2] No such file or directory: './saved_models/20230416_122752_FactorizationMachineModel(\n  (embedding): FeaturesEmbedding(\n    (embedding): Embedding(309818, 16)\n  )\n  (linear): FeaturesLinear(\n    (fc): Embedding(309818, 1)\n  )\n  (fm): FactorizationMachine()\n)_model.pt'